# Running an existing model of pose ID

In [ ]:
# Just run in case this have not been installed previuosly
!pip install mediapipe opencv-python pandas scikit-learn

In [8]:
import mediapipe as mp
import cv2 

# From step 2
import os
import numpy as np

# From step 3
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline # allows you to build a ML pipeline
from sklearn.preprocessing import StandardScaler # Normalizes your data

# Diferent algorithms for diversifing your model training (?)
from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# From step 4
from sklearn.metrics import accuracy_score # accuracy metrics
import pickle

In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [10]:
# open the model that was saved
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [11]:
cap = cv2.VideoCapture(0)

#Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245,117,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,256,121), thickness=2,circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract face landmarks 
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concatenate rows
            row = pose_row + face_row
            
#             # Append class name
#             row.insert(0, class_name)
            
            # Make detections
            x = pd.DataFrame([row])
            body_language_class = model.predict(x)[0]
            body_language_prob = model.predict_proba(x)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coordinates
            display_coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                                     , [640, 480]).astype(int))
            
            cv2.rectangle(image,
                         (display_coords[0], display_coords[1]+5),
                         (display_coords[0]+ len(body_language_class)*20, display_coords[1]-30),
                         (245, 117, 16), -1)
            cv2.putText(image, body_language_class, display_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Export to .CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row)
            
        except:
            pass
        
        cv2.imshow('Raw webcam feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.06 0.94]
Winning [0.   0.06 0.94]
Winning [0.   0.06 0.94]
Winning [0.   0.06 0.94]
Winning [0.   0.06 0.94]
Winning [0.   0.05 0.95]
Winning [0.   0.06 0.94]
Winning [0.   0.06 0.94]
Winning [0.   0.03 0.97]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.04 0.95]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]


Winning [0.12 0.12 0.76]
Winning [0.12 0.12 0.76]
Winning [0.12 0.12 0.76]
Winning [0.12 0.1  0.78]
Winning [0.12 0.1  0.78]
Winning [0.12 0.11 0.77]
Winning [0.12 0.12 0.76]
Winning [0.14 0.13 0.73]
Winning [0.15 0.13 0.72]
Winning [0.15 0.08 0.77]
Winning [0.16 0.09 0.75]
Winning [0.16 0.07 0.77]
Winning [0.15 0.1  0.75]
Winning [0.15 0.13 0.72]
Winning [0.15 0.15 0.7 ]
Winning [0.14 0.15 0.71]
Winning [0.14 0.11 0.75]
Winning [0.14 0.13 0.73]
Winning [0.15 0.07 0.78]
Winning [0.15 0.07 0.78]
Winning [0.14 0.07 0.79]
Winning [0.15 0.1  0.75]
Winning [0.1  0.03 0.87]
Winning [0.08 0.03 0.89]
Winning [0.07 0.02 0.91]
Winning [0.07 0.02 0.91]
Winning [0.06 0.01 0.93]
Winning [0.05 0.01 0.94]
Winning [0.06 0.02 0.92]
Winning [0.05 0.02 0.93]
Winning [0.05 0.02 0.93]
Winning [0.08 0.03 0.89]
Winning [0.08 0.03 0.89]
Winning [0.08 0.04 0.88]
Winning [0.08 0.05 0.87]
Winning [0.08 0.04 0.88]
Winning [0.08 0.04 0.88]
Winning [0.08 0.04 0.88]
Winning [0.08 0.04 0.88]
Winning [0.08 0.04 0.88]


Winning [0.06 0.04 0.9 ]
Winning [0.06 0.05 0.89]
Winning [0.06 0.04 0.9 ]
Winning [0.06 0.04 0.9 ]
Winning [0.05 0.04 0.91]
Winning [0.05 0.06 0.89]
Winning [0.06 0.06 0.88]
Winning [0.04 0.06 0.9 ]
Winning [0.04 0.03 0.93]
Winning [0.04 0.05 0.91]
Winning [0.05 0.05 0.9 ]
Winning [0.06 0.06 0.88]
Winning [0.08 0.05 0.87]
Winning [0.08 0.05 0.87]
Winning [0.07 0.06 0.87]
Winning [0.08 0.07 0.85]
Winning [0.08 0.05 0.87]
Winning [0.09 0.05 0.86]
Winning [0.09 0.05 0.86]
Winning [0.08 0.04 0.88]
Winning [0.1  0.06 0.84]
Winning [0.1  0.05 0.85]
Winning [0.11 0.06 0.83]
Winning [0.12 0.06 0.82]
Winning [0.12 0.07 0.81]
Winning [0.12 0.08 0.8 ]
Winning [0.12 0.09 0.79]
Winning [0.11 0.08 0.81]
Winning [0.11 0.08 0.81]
Winning [0.13 0.13 0.74]
Winning [0.25 0.13 0.62]
Winning [0.41 0.11 0.48]
Winning [0.1  0.02 0.88]
Winning [0.09 0.02 0.89]
Winning [0.08 0.02 0.9 ]
Winning [0.08 0.02 0.9 ]
Winning [0.09 0.02 0.89]
Winning [0.1  0.01 0.89]
Winning [0.11 0.01 0.88]
Winning [0.12 0.02 0.86]


Winning [0.03 0.   0.97]
Winning [0.01 0.   0.99]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.02 0.05 0.93]
Winning [0.01 0.05 0.94]
Winning [0.02 0.02 0.96]
Winning [0.11 0.04 0.85]
Winning [0.14 0.04 0.82]
Winning [0.14 0.07 0.79]
Winning [0.12 0.06 0.82]
Winning [0.12 0.1  0.78]
Winning [0.12 0.17 0.71]
Winning [0.11 0.16 0.73]
Winning [0.03 0.1  0.87]
Winning [0.02 0.31 0.67]
Winning [0.01 0.29 0.7 ]
Sad [0.   0.59 0.41]
Winning [0.01 0.19 0.8 ]
Winning [0.   0.08 0.92]
Winning [0.   0.07 0.93]
Winning [0.   0.05 0.95]
Winning [0.   0.05 0.95]
Winning [0.   0.05 0.95]
Winning [0.   0.07 0.93]
Winning [0.   0.06 0.94]
Winning [0.02 0.07 0.91]
Winning [0.02 0.08 0.9 ]
Winning [0.02 0.08 0.9 ]
Winning [0.02 0.07 0.91]
Winning [0.02 0.06 0.92]
Winning [0.02 0.06 0.92]
Winning [0.02 0.07 0.91]
Winning [0.01 0.07 0.92]
Winning [0.01 0.06 0.93]
Winning [0.01 0.06 0.93]
Winning [0.02 0.06 0.92]
Winning [0.02 0.05 0.93]
Winning [0.03 0.08 0.89]
Winning [0.03 0.08 0.89]
Winn

Winning [0.02 0.17 0.81]
Winning [0.  0.2 0.8]
Winning [0.02 0.27 0.71]
Winning [0.02 0.27 0.71]
Winning [0.02 0.28 0.7 ]
Winning [0.02 0.26 0.72]
Winning [0.02 0.26 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.01 0.27 0.72]
Winning [0.02 0.26 0.72]
Winning [0.04 0.23 0.73]
Winning [0.07 0.22 0.71]
Winning [0.04 0.24 0.72]
Winning [0.03 0.24 0.73]
Winning [0.06 0.09 0.85]
Winning [0.05 0.46 0.49]
Winning [0.06 0.26 0.68]
Winning [0.06 0.4  0.54]
Winning [0.07 0.12 0.81]
Winning [0.07 0.12 0.81]
Winning [0.07 0.14 0.79]
Winning [0.07 0.14 0.79]
Winning [0.07 0.13 0.8 ]
Winning [0.06 0.22 0.72]
Winning [0.06 0.22 0.72]
Winning [0.07 0.21 0.72]
Winning [0.08 0.2  0.72]
Winning [0.07 0.22 0.71]
Winning [0.05 0.22 0.73]
Winning [0.07 0.21 0.72]
Winning [0.07 0.21 0.72]
Winning [0.07 0.21 0.72]
Winning [0.05 0.22 0.73]
Winning [0.07 0.21 0.72]
Win

...

Winning [0.   0.06 0.94]
Winning [0.   0.05 0.95]
Winning [0.   0.05 0.95]
Winning [0.   0.05 0.95]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.04 0.96]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.04 0.96]
Winning [0.   0.04 0.96]
Winning [0.   0.04 0.96]
Winning [0.   0.05 0.95]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]


Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]


Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]


Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]


Winning [0.01 0.03 0.96]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0. 0. 1.]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.01 0.01 0.98]
Winning [0.01 0.03 0.96]
Winning [0.   0.01 0.99]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0. 0. 1.]
Winning [0.   0.01 0.99]
Winning [0. 0. 1.]
Winning [0. 0. 1.]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.01 0.01 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]